In [51]:
import IPython, nussl, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math
from IPython.display import Audio, display
from scipy.fftpack import fft, ifft
 
# This line makes sure your plots happen IN the webpage you're building, instead of in separate windows.
%matplotlib inline

In [21]:
n_fft = 2048
hop_length = 512
sr = 22050

# get the signal
signal0, sr = librosa.load(path='datasets/development/blues/blues.00000.au', sr=sr)
signal1, sr = librosa.load(path='datasets/development/blues/blues.00001.au', sr=sr)
background0, sr = librosa.load(path='datasets/development/blues/blues.background.00000.au', sr=sr)
background1, sr = librosa.load(path='datasets/development/blues/blues.background.00001.au', sr=sr)
foreground0, sr = librosa.load(path='datasets/development/blues/blues.foreground.00000.au', sr=sr)
foreground1, sr = librosa.load(path='datasets/development/blues/blues.foreground.00001.au', sr=sr)
harmonic0, sr = librosa.load(path='datasets/development/blues/blues.harmonic.00000.au', sr=sr)
harmonic1, sr = librosa.load(path='datasets/development/blues/blues.harmonic.00001.au', sr=sr)
percussive0, sr = librosa.load(path='datasets/development/blues/blues.percussive.00000.au', sr=sr)
percussive1, sr = librosa.load(path='datasets/development/blues/blues.percussive.00001.au', sr=sr)

In [22]:
def meanAndStandardDeviationCentroid(signal, sample_rate, n_fft, hop_length):
    """
    computes the mean and standard deviation centroid, given an audio vector of an audio file
    """
    centroid = librosa.feature.spectral_centroid(signal, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)
    mean = np.mean(centroid)
    standard_dev = np.std(centroid)
    return mean, standard_dev
    

In [23]:
def meanAndStandardDeviationRolloff(signal, sample_rate, n_fft, hop_length):
    """
    computes the mean and standard deviation rolloff, given an audio vector of an audio file
    """
    rolloff = librosa.feature.spectral_rolloff(signal, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)
    mean = np.mean(rolloff)
    standard_dev = np.std(rolloff)
    return mean, mean

In [41]:
###     ask Preem about mean vs standard dev
def ZeroCrossings(signal):
    """
    computes zero crossings value of a signal
    """
    zero_crossings = librosa.zero_crossings(signal)
    
    return np.sum(zero_crossings)
    

In [43]:
### ask Preem about flux
def flux(signal):
    return


In [44]:
### ask about speed of auto-correlation
def autoCorrelation(signal):
    """
    gets the auto-correlation of a given signal
    """
    return librosa.autocorrelate(signal)

In [52]:
def acorr(x):
    """
    Takes a 1D numpy array and returns the autocorrelation. 
    
    Input Parameter:
    ----------------
    x: 1D numpy array of length Lx
    
    Ouput Parameter:
    ----------------
    x_acorr: 1D numpy array of length x_len containing the values of the autocorrelation function of x
    
    Note: the actual length of autocorrelation function is (2*x_len)-1, but since this function is symmetric
          we can cut off (x_len)-1 samples and return one side of length x_len without losing information. 
    """
    
    x_len=np.size(x)
    x_pad=np.concatenate([x,np.zeros(x_len-1)])
    x_acorr=ifft(np.abs(fft(x_pad))**2).real
    x_acorr=x_acorr[0:x_len]
    
    x_acorr=x_acorr/(np.arange(x_len)[::-1]+1)  # normalize by the number of nonzero additions
            
    return x_acorr

In [53]:
x = acorr(signal0)

In [54]:
y = autoCorrelation(signal0)

In [57]:
print x
print y

[ 0.01979324  0.01908897  0.01759472 ..., -0.00063689 -0.00075636
 -0.00047006]
[  1.30990469e+04   1.26329502e+04   1.16440469e+04 ...,  -1.91317627e-03
  -1.51599583e-03  -4.74037020e-04]
